In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_data = pd.read_csv("C://Users/Furkan/Desktop/onlinedatasciencechallenge/train.csv")
test_input = pd.read_csv("C://Users/Furkan/Desktop/onlinedatasciencechallenge/test_input.csv")

In [ ]:
train_data.head()

In [ ]:
test_input.head()

In [ ]:
y = train_data.iloc[:,2]
#dates = train_data.iloc[:,1]
train_data = train_data.iloc[:,3:]
train_data = pd.concat([train_data,y],axis=1)
#train_data = pd.concat([dates,train_data],axis=1)
test_data = test_input.iloc[:,2:]

In [ ]:
train_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

### 'CONTPAIDAMNT07' kolonunun test veri setinde eksik olduğunu görüyoruz ve verilerden atıyoruz.

In [ ]:
print(train_data.columns)
print(test_data.columns)

In [ ]:
train_data.drop('CONTPAIDAMNT07', axis=1, inplace=True)
train_data.head()

In [ ]:
test_data.head()

In [ ]:
for col in train_data.columns:
    print(col, str(round(100* train_data[col].isnull().sum() / len(train_data), 2)) + '%')

### "BLNAMNT03", "EFTAMNTSUM12","DEBTAVG00" "LASTAUTOPAYTIME", "TIMEDEPAVG12" kolonlarını eksik veriden dolayı atmak zorundayız.
## Veri kaybının %50'den fazla olduğu kolonlar

In [ ]:
train_data.drop(["BLNAMNT03","EFTAMNTSUM12","DEBTAVG00","LASTAUTOPAYTIME","TIMEDEPAVG12"], axis=1, inplace=True)
test_data.drop(["BLNAMNT03","EFTAMNTSUM12","DEBTAVG00","LASTAUTOPAYTIME","TIMEDEPAVG12"], axis=1, inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)

train_data.describe()

In [ ]:
train_data = train_data.dropna(how="any",axis=0)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.fillna(train_data.mean(),inplace=True)
test_data.fillna(test_data.mean(),inplace=True)

In [ ]:
train_data.describe().T

### Outlier değerlerden kurtulmak için IsolationForest kullanıyoruz.

In [ ]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.25)
yhat = iso.fit_predict(train_data)

In [ ]:
mask = yhat != -1

train_inlier = train_data.iloc[mask,:]

In [ ]:
train_inlier.describe().T

In [ ]:
train_inlier.corr()

In [ ]:
train_data.loc[train_data.ADDCONTAMNT != 0, "ADDCONTAMNT"] = 1
#train_inlier.loc[train_inlier.ADDCONTAMNT != 0, "ADDCONTAMNT"] = 1

In [ ]:
train_data.head()
#train_inlier.head()

In [ ]:
y_data = train_data.iloc[:,-1]
#y_inlier = train_inlier.iloc[:,-1]


In [ ]:
train_data.drop("ADDCONTAMNT",axis=1, inplace=True)
#train_inlier.drop("ADDCONTAMNT",axis=1, inplace=True)

# Modelling

first classification

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, y_data, test_size=0.3, random_state=8)
#x_train, x_test, y_train, y_test = train_test_split(train_inlier, y_inlier, test_size=0.3, random_state=8)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 0)
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,logreg_pred)))
print("RMSE: " + str(mean_squared_error(y_test,logreg_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,logreg_pred))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,knn_pred)))
print("RMSE: " + str(mean_squared_error(y_test,knn_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,knn_pred))

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel="linear")
svc.fit(x_train,y_train)
svc_pred = svc.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,svc_pred)))
print("RMSE: " + str(mean_squared_error(y_test,svc_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,svc_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB
gnb.fit(x_train, y_train)
gnb_pred = gnb.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,gnb_pred)))
print("RMSE: " + str(mean_squared_error(y_test,gnb_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,gnb_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth= 7, criterion = "entropy")
dtc.fit(x_train,y_train)
dtc_pred = dtc.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,dtc_pred)))
print("RMSE: " + str(mean_squared_error(y_test,dtc_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,dtc_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100, criterion = "gini")
rfc.fit(x_train,y_train)
rfc_pred = rfc.predict(x_test)
#rfc_pred2 = rfc.predict(test_data)

print("R^2 Score: " + str(r2_score(y_test,rfc_pred)))
print("RMSE: " + str(mean_squared_error(y_test,rfc_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,rfc_pred))

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(x_train, y_train)
xg_pred = classifier.predict(x_test)
#xg_pred2 = classifier.predict(test_data)

print("R^2 Score: " + str(r2_score(y_test,xg_pred)))
print("RMSE: " + str(mean_squared_error(y_test,xg_pred, squared=False)))
print("Confusion matrix:")
print(confusion_matrix(y_test,xg_pred))


# Modelling

Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)
lr_pred = lr.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score

print("R^2 Score: " + str(r2_score(y_test,lr_pred)))
print("RMSE: " + str(mean_squared_error(y_test,lr_pred, squared=False)))


In [ ]:
from sklearn.svm import LinearSVR
kernel = ["linear", "poly", "rbf", "sigmoid", "precomputed"]

svr = LinearSVR()
svr.fit(x_train,y_train)
svr_pred = svr.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,svr_pred)))
print("RMSE: " + str(mean_squared_error(y_test,svr_pred, squared=False)))


In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=None, criterion = "mse", min_samples_split = 2,min_samples_leaf= 7)
dtr.fit(x_train,y_train)
dtr_pred = dtr.predict(x_test)
dtr2_pred = dtr.predict(test_data)

print("R^2 Score: " + str(r2_score(y_test,dtr_pred)))
print("RMSE: " + str(mean_squared_error(y_test,dtr_pred, squared=False)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 100, criterion = "mse")
rfr.fit(x_train,y_train)
rfr_pred = rfr.predict(x_test)

print("R^2 Score: " + str(r2_score(y_test,rfr_pred)))
print("RMSE: " + str(mean_squared_error(y_test,rfr_pred, squared=False)))

In [ ]:
output = pd.DataFrame({'Id': test_input["CUSTNBR"], 'Predicted': dtr2_pred})
output.to_csv('denemesub.csv', index=False)
print("Your submission was successfully saved!")